In [4]:
import pandas as pd
import numpy as np
import os
import pathlib
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import tensorflow as tf

from bokeh.plotting import figure, show, output_notebook, output_file, save
from bokeh.models import LinearAxis, Range1d, ColumnDataSource, HoverTool
from bokeh.embed import file_html
from bokeh.layouts import row, gridplot
from bokeh.palettes import Spectral11 as palette
from bokeh.io import curdoc, export_png
from scipy.optimize import curve_fit
from tensorflow import keras
from tensorflow.keras import layers

output_notebook()

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Get Data

  



In [5]:
def find_file_and_get_raw_datas(folder_path, file_extension):
  # 폴더 안 파일을 file_list에 저장
  file_list = os.listdir(folder_path)

  # file_list 중 file_extension으로 지정해준 확장자인 것만 all_file_list에 저장
  all_file_list = []
  for file in file_list:
    if file.endswith(file_extension):
      all_file_list.append(file)

  # all_file_list 정렬
  all_file_list = sorted(all_file_list)
  
  # all_file_list로 해당 파일 정보 df 형태로 가져온 후 리스트에 저장해 반환
  log_datas = []
  for file in all_file_list:
    log_datas.append(pd.read_csv(folder_path + "/" + file, sep=',', encoding="cp949"))
  return log_datas
  

# Data post processing



In [6]:
df_datas = pd.read_csv("/content/drive/MyDrive/Robostar/Model_Fitting_data/J1_Friction_Law.csv")
def data_post_processing(log_datas):
  
  TINY = 1e-6
  results = []

  for x in range(len(log_datas)):
    df_datas = log_datas[x]

    # Remove Section Zero
    modified_df = df_datas[df_datas["section_block"] != 0]
    modified_df = modified_df.reset_index(drop=True)

    # Calculate Friction Torque
    for i in range(6):
      modified_df["trq_fric" + str(i+1)] = modified_df["motor_trq" + str(i+1)] - modified_df["cal_trq" + str(i+1)]

    # Find Constant Speed
    modified_df["joint_acc_ref" + str(x + 1)] = modified_df["joint_vel_ref" + str(x + 1)].diff() / 0.002
    modified_df = modified_df[np.abs(modified_df['joint_acc_ref' + str(x + 1)]) < TINY]
    modified_df = modified_df.reset_index(drop=True)

    # Delete Velocity zero
    modified_df = modified_df[np.abs(modified_df["joint_vel_ref" + str(x + 1)]) > TINY]     # joint_vel_ref1
    modified_df = modified_df.reset_index(drop=True)

    # Average Torque
    vel, ave_fric = [], []
    for i in range(1, max(modified_df['section_block'])):
      section_df = modified_df[modified_df['section_block'] == i]
      section_df = section_df[:][len(section_df)//3:-len(section_df)//3]
      vel.append(section_df.iloc[0]['joint_vel_ref' + str(x + 1)])
      ave_fric.append(section_df['trq_fric' + str(x + 1)].mean())

    data = {'vel': vel, 'ave_fric': ave_fric}
    result_df = pd.DataFrame(data)
    result_df = result_df.sort_values(by='vel')

    result_df.reset_index(drop=True, inplace=True)
    results.append(result_df)
  return results

# CTO Model Function

In [7]:
def func(x, a1, a2, a3, a4, r):
  'a1*vel + a2*vel**3 + a3*tanh(r*vel) + a4*((r*x)/(0.005+abs(r*x)))'
  return a1 * x + a2 * x ** 3 + a3 * np.tanh(r*x) + a4*((r*x)/(0.0005+abs(r*x)))

# Model Fitting & Result

In [8]:
def model_fitting(processing_results):
  popt_pcov_list = []
  
  results = []
  for i in range(len(processing_results)):
    x = np.array(processing_results[i]["vel"].values)
    y = np.array(processing_results[i]["ave_fric"].values)

    popt, pcov = curve_fit(func, x, y)

    modelPredictions = func(x, *popt)
    absError = modelPredictions - y
    SE = np.square(absError)
    MSE = np.mean(SE)
    RMSE = np.sqrt(MSE)
    Rsquared = 1.0 - (np.var(absError) / np.var(y))

    #print('Parameters:', popt)
    #print('RMSE:', RMSE)
    #print('R-squared:', Rsquared)

    result = {'x' : x, 'y': y, 'popt' : popt}

    results.append(result)
  return results

# Graph

In [9]:
def start():
  folder_path = "/content/drive/MyDrive/Robostar/Model_Fitting_data"

  df_datas = find_file_and_get_raw_datas(folder_path, '.csv')
  processing_results = data_post_processing(df_datas)
  fitting_results = model_fitting(processing_results)

  plots = []
  for i in range(len(fitting_results)):
    curdoc().theme = "caliber"
    TOOLS = "pan, wheel_zoom, box_select, lasso_select, reset, save, hover"
    TOOLTIPS = [
      ("index", "$x"),
      ("y_value", "$y)"),
    ]
    # x : fitting_results[i]['x'], y : fitting_results[i]['y'] or func(fitting_results[i]['x'], *fitting_results[i]["popt"])
    p1 = figure(title="J" + str(i + 1), x_axis_label="vel (deg/s)", y_axis_label="torque(N/m)", plot_width=1200, plot_height=900, tools=TOOLS, tooltips=TOOLTIPS)

    p1.line(x = fitting_results[i]['x'], y = func(fitting_results[i]['x'], *fitting_results[i]["popt"]), color = palette[0], legend_label = 'Model', line_width = 5, line_alpha = 0.3)
    p1.scatter(x = fitting_results[i]['x'], y = fitting_results[i]['y'], legend_label = 'Data', marker = 'o', color = palette[-1], fill_color=palette[-1])

    p1.legend.location = "top_right"
    p1.legend.click_policy = "hide"
    p1.background_fill_color="#f0f0f0"
    p1.grid.grid_line_color="#f5f5f5"
    plots.append(p1)

  plot = gridplot(plots, ncols=3, toolbar_location="right", plot_width=400, plot_height=300)
  show(plot)
  #output_file(folder_path + "/" + "results_graph" + '.html')
  #save(plot)

  fw = open(folder_path + "/" + "results_popt.txt", 'w')
  for i in range(len(fitting_results)): # popt 저장
    popt_str = ""
    fw.write("<J" + str(i + 1) + ">\n")
    for popt in fitting_results[i]["popt"]:
      popt_str += str(popt)  + ","
    fw.write(popt_str[:-1] + "\n")
  fw.close()
    

In [10]:
def norm(x, train_stats):
  return (x - train_stats["mean"]) / train_stats["std"]
    
def build_model(train_dataset):
  model = keras.Sequential([
                            layers.Dense(64, activation="relu", input_shape=[len(train_dataset.keys())]),
                            layers.Dense(64, activation="relu"),
                            layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss="mse", optimizer=optimizer, metrics=["mae", "mse"])
  return model

def tensorflow_start():
  folder_path = "/content/drive/MyDrive/Robostar/Model_Fitting_data"

  df_datas = find_file_and_get_raw_datas(folder_path, '.csv')
  processing_results = data_post_processing(df_datas)
  fitting_results = model_fitting(processing_results)

  # 각 축에 대한 개별 학습 및 그래프 
  plots = []
  for i in range(len(fitting_results)):
    dataset = pd.DataFrame()
    dataset['x'] = fitting_results[i]['x']
    dataset['y'] = fitting_results[i]['y']

    train_dataset = dataset.sample(frac=0.8, random_state=0)
    test_dataset = dataset.drop(train_dataset.index)

    train_stats = train_dataset.describe()
    train_stats.pop('y')
    train_stats = train_stats.transpose()

    train_labels = train_dataset.pop('y')
    test_labels = test_dataset.pop('y')

    normed_x = norm(dataset, train_stats)
    normed_train_data = norm(train_dataset, train_stats)
    normed_test_Data = norm(test_dataset, train_stats)

    model = build_model(train_dataset)
    EPOCHS = 5000
    early_stop = keras.callbacks.EarlyStopping(monitor="val_loss", patience=100)
    history = model.fit(
    normed_train_data, train_labels,
    epochs=EPOCHS, validation_split=0.2, verbose=0, callbacks=[early_stop])

    prediction = model.predict(normed_x['x']).flatten()
    ## 그래프
    curdoc().theme = "caliber"
    TOOLS = "pan, wheel_zoom, box_select, lasso_select, reset, save, hover"
    TOOLTIPS = [
      ("index", "$x"),
      ("y_value", "$y)"),
    ]
    # x : fitting_results[i]['x'], y : fitting_results[i]['y'] or func(fitting_results[i]['x'], *fitting_results[i]["popt"])
    p1 = figure(title="J" + str(i + 1), x_axis_label="vel (deg/s)", y_axis_label="torque(N/m)", plot_width=1200, plot_height=900, tools=TOOLS, tooltips=TOOLTIPS)

    p1.line(x = fitting_results[i]['x'], y = func(fitting_results[i]['x'], *fitting_results[i]["popt"]), color = palette[0], legend_label = 'curve_Model', line_width = 5, line_alpha = 0.7)
    p1.line(x = fitting_results[i]['x'], y = prediction, color = palette[3], legend_label = 'tensor', line_width = 5, line_alpha = 0.7)
    p1.scatter(x = fitting_results[i]['x'], y = fitting_results[i]['y'], legend_label = 'Data', marker = 'o', color = palette[-1], fill_color=palette[-1])

    p1.legend.location = "top_right"
    p1.legend.click_policy = "hide"
    p1.background_fill_color="#f0f0f0"
    p1.grid.grid_line_color="#f5f5f5"
    plots.append(p1)
    
  plot = gridplot(plots, ncols=3, toolbar_location="right", plot_width=1200, plot_height=600)
  show(plot)


In [11]:
# J1 ~ J6 데이터 추가 후 학습 및 예측
def J1_J6_tensorflow_start():
  folder_path = "/content/drive/MyDrive/Robostar/Model_Fitting_data"

  df_datas = find_file_and_get_raw_datas(folder_path, '.csv')
  processing_results = data_post_processing(df_datas)
  fitting_results = model_fitting(processing_results)

  # 축 상관없이 x 데이터 및 y 데이터 저장
  # J1 ~ J6 해당

  dataset = pd.DataFrame()
  list_x, list_y = [], []
  for i in range(0, len(fitting_results)):
    for j in range(0, len(fitting_results[i]['x'])):
      list_x.append(fitting_results[i]['x'][j])
      list_y.append(fitting_results[i]['y'][j])
  df_x, df_y = pd.DataFrame(list_x), pd.DataFrame(list_y)
  df_x['x'], df_y['y'] = pd.DataFrame(list_x), pd.DataFrame(list_y)
  dataset['x'], dataset['y'] = df_x['x'], df_y['y']

  # 전체 데이터 train, test로 분류
  train_dataset = dataset.sample(frac=0.8,random_state=0)
  test_dataset = dataset.drop(train_dataset.index)

  train_stats = train_dataset.describe()
  train_stats.pop('y')
  train_stats = train_stats.transpose()
  
  train_labels = train_dataset.pop('y')
  test_labels = test_dataset.pop('y')

  normed_train_data = norm(train_dataset, train_stats)
  normed_test_data = norm(test_dataset, train_stats)

  # 모델 학습
  EPOCHS = 5000
  model = build_model(train_dataset)
  early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
  history = model.fit(normed_train_data, train_labels, epochs=EPOCHS,
                      validation_split = 0.2, verbose=0, callbacks=[early_stop])
  
  loss, mae, mse = model.evaluate(normed_test_data, test_labels, verbose=2)
  print("테스트 세트의 평균 절대 오차: {:5.2f} y".format(mae))
  print(normed_test_data)
  # 학습한 모델로 각 축별 예측
  for i in range(len(fitting_results)):
    # 한 축 x 데이터 및 y 데이터
    sep_dataset = pd.DataFrame()
    sep_dataset['x'] = fitting_results[i]['x']
    sep_dataset['y'] = fitting_results[i]['y']

    sep_stats = sep_dataset.describe()
    sep_labels = sep_dataset.pop('y')
    sep_stats = sep_stats.transpose()



    sep_normed_data = norm(sep_dataset, sep_stats)
    print(sep_normed_data)
    predictions = model.predict(sep_normed_data['x']).flatten()

    plt.scatter(sep_labels, predictions)
    plt.xlabel('True Values [y]')
    plt.ylabel('Predictions [y]')
    plt.axis('equal')
    plt.axis('square')
    plt.xlim([0,plt.xlim()[1]])
    plt.ylim([0,plt.ylim()[1]])
    _ = plt.plot([-100, 100], [-100, 100])



# J1_J6_tensorflow_start()



# PPTX

In [12]:
!pip install python-pptx
import datetime
from pytz import timezone
from pptx import Presentation
from pptx.util import Inches

def make_pptx():
  prx = Presentation()

  title_slide_layout = prx.slide_layouts[0]
  slide = prx.slides.add_slide(title_slide_layout)
  title = slide.placeholders[0]

  title.text = str(datetime.datetime.now(timezone("UTC")).astimezone(timezone("Asia/Seoul")).strftime("%Y-%m-%d %H:%M:%S")) + " Report"

  subtitle = slide.placeholders[1]
  subtitle.text = "DNN Graph Report"

  img_path_list = []
  folder_path = "/content/drive/MyDrive/Robostar/PPT_test"
  file_list = os.listdir(folder_path)

  for file in file_list:
    if file.endswith(".png"):
      img_path_list.append(file)
  img_path_list = sorted(img_path_list)

  for image in img_path_list:
    print(image)
    blank_slide_layout = prx.slide_layouts[1]
    slide = prx.slides.add_slide(blank_slide_layout)

    slide.placeholders[0].text = image + " Graph"
    slide.placeholders[1].text_frame.text = "bullet slide layout example"
    p = slide.placeholders[1].text_frame.add_paragraph()
    p.text = "Text frame example for first bullet"
    p.level = 1

    p = slide.placeholders[1].text_frame.add_paragraph()
    p.text = "Text frame example for second bullet"
    p.level = 2

    left, top = Inches(0.5), Inches(3.5)
    height = Inches(4)
    pic = slide.shapes.add_picture(folder_path + '/' + image, left, top, height=height)
    print(left)
  prx.save("/content/drive/MyDrive/Robostar/PPT_test/test_ppt.pptx")


In [13]:
 start()

/usr/local/lib/python3.7/dist-packages/scipy/optimize/minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


In [14]:
tensorflow_start()

/usr/local/lib/python3.7/dist-packages/scipy/optimize/minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


In [15]:
make_pptx()

bokeh_plot (1).png
457200
bokeh_plot (2).png
457200
bokeh_plot (3).png
457200
bokeh_plot (4).png
457200
bokeh_plot (5).png
457200
bokeh_plot.png
457200
